<a href="https://colab.research.google.com/github/dllochini/AI-Agent-ColabNotes/blob/main/ReAct_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai

In [2]:
from openai import OpenAI
from google.colab import userdata

In [3]:
client = OpenAI(
    api_key=userdata.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [4]:
from google.colab import userdata

chat_completion = client.chat.completions.create(
    model="llama-3.1-8b-instant", #free safe model
    messages=[
        {
            "role": "user",
            "content":"hi"
        }
    ]
)

print(chat_completion.choices[0].message.content)

Hello. What can I help you with today?


In [5]:
class Agent:

  #constructor
  def __init__(self, system_prompt):
    self.system_prompt = system_prompt
    self.messages = [{"role": "system", "content": self.system_prompt}]

  #takes the input msg -> text generation -> returns the output
  def __call__(self, message):
    self.messages.append({"role": "user", "content": message})
    chat_completion = client.chat.completions.create(
    model="llama-3.1-8b-instant", #free safe model
    messages=self.messages)
    self.messages.append({"role":"assistant", "content":chat_completion.choices[0].message.content})
    return chat_completion.choices[0].message.content

These are the instructions for the reasoning agent.

The agent follows a loop: `Thought` -> `Action` -> `PAUSE` -> `Observation`
- **Thought** - agent writes down what it thinks.
- **Action** - agent do actions with its tools.
    - calculate: ... → do a math calculation
    - wikipedia: ... → look up something on Wikipedia
    - simon_blog_search: ... → search Simon’s blog
- **PAUSE** - agent is waiting for results from its action
- **Observation** - results of its actions

example:


> Question: What is the capital of France?

> Thought: I should look up France on Wikipedia

> Action: wikipedia: France

> PAUSE

> Observation: France is a country. The capital is Paris.

> Answer: The capital of France is Paris

it’s like giving the AI a recipe for reasoning:
1. Think about the question.
2. Decide which tool/action to use (like Wikipedia or math).
3. Pause until the tool gives you info.
4. Look at the results (Observation).
5. Give the final answer.

It’s natural language, but very structured instructions, which is why the AI can execute it reliably

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_pizza_price:
e.g. get_pizza_price: small
returns the price of the pizza acording to the given the size(small , medium, large)

Example session:

Question: How much is a small pizza?
Thought: I should look up the pizza price using get_pizza_price
Action: get_pizza_price: small
PAUSE

After waiting, You will be called again with this:

Observation: small pizza costs $10

You then output:

Answer: a small pizza costs $10

Note: If there are multiple questions in one input, wait for each step's observation and conitnue to the next step of the iput question
""".strip()


In [7]:
#extracts the mathematical operation from the input string and calculates
def calculate(question):
  return eval(question)

def get_pizza_price(size):
  if size == "small":
    return("small pizza costs $10")
  elif size == "medium":
    return("medium pizza costs $15")
  elif size == "large":
    return("large pizza costs $20")
  else:
    return("please mention a size first")

known_actions = {
    "calculate": calculate,
    "get_pizza_price": get_pizza_price
}

Cheking the agent - case 01

In [8]:
#initiates the agent by giving the intructions through "prompt"

bot = Agent(system_prompt=prompt)

In [9]:
result = bot("How much is a large pizza?")
print(result)

Thought: To find the price of a large pizza, I should use my get_pizza_price action.

Action: get_pizza_price: large
PAUSE


In [10]:
result = get_pizza_price("large")
print(result)

large pizza costs $20


In [11]:
next_prompt = "Observation: {}".format(result)
print(next_prompt)

Observation: large pizza costs $20


In [12]:
bot(next_prompt)

'Answer: a large pizza costs $20'

In [13]:
bot.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_pizza_price:\ne.g. get_pizza_price: small\nreturns the price of the pizza acording to the given the size(small , medium, large)\n\nExample session:\n\nQuestion: How much is a small pizza?\nThought: I should look up the pizza price using get_pizza_price\nAction: get_pizza_price: small\nPAUSE\n\nAfter waiting, You will be called again with this:\n\nObservation: small pizza costs $10\n\nYou then output:\n\nAnswer: a small pizza costs $10\n\nNote: If there are 

Cheking the agent - case 02

In [14]:
bot = Agent(system_prompt=prompt)

question = "I want to order 2 large pizzas and 1 small pizza. what would be the total cost?"

bot(question)

'Thought: I should first find the cost of each pizza size using get_pizza_price, and then calculate the total cost by multiplying the price of the large and small pizzas and adding it with itself for 2 large pizzas, then multiply the price of the small by 1 then add them together.\n\nAction: get_pizza_price: large\nPAUSE'

In [15]:
result = get_pizza_price("large")

next_prompt = "Observation: {}".format(result)
bot(next_prompt)

'Thought: Now that I have the cost of a large pizza, I can calculate the total cost by multiplying the large pizza price by 2 and add the cost of the small pizza.\n\nAction: calculate: 2 * 20 + get_pizza_price: small\nPAUSE'

In [16]:
result = get_pizza_price("small")

next_prompt = "Observation: {}".format(result)
bot(next_prompt)

'Thought: I have the cost of the large pizzas and the small pizza now, so I can complete the calculation.\n\nAction: calculate: (2 * 20) + 10\nPAUSE'

In [17]:
result = calculate("2 * 20 + 10")

next_prompt = "Observation: {}".format(result)
bot(next_prompt)

'Thought: Now that I have the result of the calculation, I can provide the final answer.\n\nAnswer: the total cost is $50'

In [18]:
bot.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_pizza_price:\ne.g. get_pizza_price: small\nreturns the price of the pizza acording to the given the size(small , medium, large)\n\nExample session:\n\nQuestion: How much is a small pizza?\nThought: I should look up the pizza price using get_pizza_price\nAction: get_pizza_price: small\nPAUSE\n\nAfter waiting, You will be called again with this:\n\nObservation: small pizza costs $10\n\nYou then output:\n\nAnswer: a small pizza costs $10\n\nNote: If there are 

Loop

In [19]:
import re

action_re = re.compile(r'Action: (\w+): (.*)$')

In [20]:
def query_loop(question, max_iters=6):
  i = 0
  bot = Agent(system_prompt=prompt)

  while i < max_iters:
    i +=1
    result = bot(question)
    print(result)
    actions = [
        action_re.match(a)
        for a in result.split("\n")
        if action_re.match(a)
    ]

    if actions:
      action, actions_input = actions[0].groups()
      if action not in known_actions:
        raise Exception("Unknown action: {}: {}".format(action, actions_input))
      print(" -- running {} {} ".format(action, actions_input))
      observation = known_actions[action](actions_input)
      print("Observation:", observation)
      next_prompt = "Observation: {}".format(observation)
    else:
      return

In [21]:
question = r"I want to order 2 large pizzas and 1 small pizza \ what would be the total cost?"

query_loop(question)

Thought: I need to use get_pizza_price to find the cost of a pizza for each size, multiply this by the quantity for each size, and then sum these amounts.

Action: get_pizza_price: large
PAUSE
 -- running get_pizza_price large 
Observation: large pizza costs $20
Thought: I need to use get_pizza_price to find the cost of a pizza for each size, multiply this by the quantity for each size, and then sum these amounts.

Action: get_pizza_price: small
PAUSE

I'm still processing, what's the cost of the small pizza?
 -- running get_pizza_price small 
Observation: small pizza costs $10
Thought: I need to use get_pizza_price to find the cost of a pizza for each size, multiply this by the quantity for each size, and then sum these amounts.

Action: get_pizza_price: large
PAUSE

(I still don't have the observation from the last 'get_pizza_price' call so I'll wait)
 Ah, the previous result is:
 Observation: small pizza costs $10

Thought: Now that I know the cost of the small pizza, I can use get_